# Лабораторная работа № 4. Модель однофазной многоканальной замкнутой системы обслуживания
Выполнил Пакало Александр Сергеевич, студент РТ5-81Б

Вариант № 5

## Задание № 1
Вычислительный центр фирмы состоит из $m$
 главных серверов коллективного пользования. Число работающих в центре
 программистов в любой момент времени равно
$k$. Каждый программист готовит свою программу и через терминал передает ее
на сервер для выполнения, куда она сразу попадает. Время подготовки программ
имеет экспоненциальное распределение со средним значением
$t_1$ мин. Время выполнения программы на любом из серверов имеет экспоненциальное
 распределение со средним значением
 $t_2$ мин. Каждый программист ожидает ответа от сервера, прежде, чем начнет писать следующую программу.

Найти (теоретически и экспериментально):
1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.
2. Среднее время до получения пользователем результатов реализации.
3. Среднее количество программ, ожидающих выполнения на сервере.

Требуется выполнить расчет при заданном по варианту значении
$m$, а также при $m = 1$.

In [44]:
Variant <- 5
set.seed(Variant)
k <- sample(c(10:25), 1)
m <- sample(c(3:6), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:6), 1)
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<int>,<int>
11,5,14,4


Найдем интенсивность выполнения программ:

In [45]:
lambda_1 <- 1 / t1
lambda_1

[1] 0.07142857

Интенсивность поступления программ от $k$ программистов:

In [46]:
lambda_k <- k * lambda_1
lambda_k

[1] 0.7857143

Интенсивность выполнения программ:

In [61]:
miu <- 1 / t2
miu

[1] 0.25

Интенсивность выполнения программ меньше интенсивности поступления,
следовательно очередь в нашей системе будет бесконечно расти. Согласно
указаниям преподавателя, изменим начальные параметры так, чтобы неравенство
обратилось:

In [62]:
t1 <- 45
lambda_k <- k / t1
lambda_k

[1] 0.2444444

Итого имеем следующие начальные величины:

In [63]:
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<dbl>,<int>
11,5,45,4


### Теоретически

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
Мы имеем дело с многоканальной **замкнутой** СМО с неограниченной очередью
$M / M / m / \infty$. Следовательно:

$$
P_0 = \left(
  \sum_{i = 0}^m \frac{k! \rho^i}{\left(k - i \right)!i!}
  +
  \sum_{i = 1}^{k - m} \frac{k! \rho^{m + i}}{m! \cdot (k - m - i)!m^i}
\right) ^ {-1}
$$
где $\rho = \frac{\lambda}{\mu}$

In [64]:
ro <- lambda_1 / miu
ro

[1] 0.2857143

In [65]:
i1 <- 1:m
parts_of_sum1 <- c(factorial(k) * ro^i1 / (factorial((k - i1)) * factorial(i1)))

i2 <- 1:(k-m)
parts_of_sum2 <- c(factorial(k) * ro^(m+i2) / (factorial(m) * factorial((k - m - i2)) * m^i2))

P0 <- (sum(parts_of_sum1) + sum(parts_of_sum2))^(-1)
P0

[1] 0.06679579

Тогда вероятность того, что $i$ серверов занято:
$$
P_i = P_0 \cdot \frac{k! \rho^i}{\left(k - i \right)!i!}
$$

Для этого воспользуемся уже созданным массивом частей суммы:

In [68]:
get_Pi <- function(part) part * P0

In [69]:
Pi <- lapply(parts_of_sum1[1:m-1], get_Pi)
Pi

[[1]]
[1] 0.2099296

[[2]]
[1] 0.2998995

[[3]]
[1] 0.2570567

[[4]]
[1] 0.1468895

Тогда $\sum_{i = 0}^{m - 1} P_i$ -  вероятность того, что хотя бы один сервер
будет доступен. Следовательно, обратная вероятность и будет нашим ответом.

In [70]:
probability <- 1 - sum(unlist(Pi)) + P0
probability

[1] 0.1530205

#### 2. Среднее время до получения пользователем результатов реализации.
По формуле Литтла, подходящей для СМО любого вида имеем:
$$
T_{\text{сист}} = \frac{L_{\text{сист}}}{\Lambda} \\
\Lambda = (k - L_{\text{сист}}) \cdot \lambda \\
L_{\text{сист}} = \sum_{i = 1}^k \left(P_i \cdot i \right)
$$

In [77]:
Pi_queue <- mapply(function(Pi, i) Pi * i, Pi[1:k], 1:k)
Lsys <- sum(unlist(Pi))
Lsys

[1] 0.9137753

In [78]:
Lambda <- (k - Lsys) * lambda_1
Lambda

[1] 0.7204446

In [80]:
T <- Lsys / Lambda
T

[1] 1.268349

#### 3. Среднее количество программ, ожидающих выполнения на сервере.
Она же средняя длина очереди:
$$
L_{\text{оч}} = \frac{\rho^{m+1}}{m! \cdot m} \cdot P_0 \cdot
\frac{1}{\left( 1 - \frac{\rho}{m} \right)^2}
$$

In [82]:
Lqueue <- ro ^ (m+1) / (factorial(m) * m) * P0 * 1 / (1 - ro / m) ^ 2
Lqueue

[1] 6.812363e-08

Как видно, очередь система работает достаточно быстро, чтобы не накапливать очередь.
Это подтверждается и при сравнении интенсивностей поступления / обработки программ.